# _Scala Fundamentals_
# 7. For Comprehensions

_This module was originally developed by Marcus Henry, Jr. It has been updated and modified since he wrote it._

## More than just a `for` loop

Let's start by considering a simple sequence of numbers.

In [ ]:
val numbers = 1 to 21 by 3

We can use the `for` statement to loop over this sequence:

In [ ]:
for (i <- 0 until numbers.size) println(i)

val copy = collection.mutable.Buffer.empty[Int]
for (i <- 0 until numbers.size) {
  val item = numbers(i)
  copy += item
}

println(numbers)
println(copy)

### Indexing isn't necessary

We can iterate directly over the elements of the collection.

In [ ]:
val doubled = collection.mutable.Buffer.empty[Int]
for (num <- numbers) {
  doubled += num * 2
}

println(doubled)

### No need for nested loops

The `for` loop supports nested loops in a pleasing syntax:

In [ ]:
val moreNumbers = 10 to 1 by -3

// Syntax quirk: You have to use curly braces, instead of
// parentheses, if you want multiple lines without semicolons.
for { m <- numbers.take(3)
      n <- moreNumbers } {
  println(s"$m * $n = ${m * n}")
}

In [ ]:
val evenMoreNumbers = (1 to 10).map { _ => scala.util.Random.nextInt(20) + 1 }
var bigNumber = BigInt(1)
for { i <- numbers
      j <- moreNumbers
      k <- evenMoreNumbers }
  bigNumber = bigNumber * i * j * k

println(bigNumber)

### It's sweet syntactic sugar

- This kind of `for` loop is really syntactic sugar for a `foreach` function.
- `foreach` is defined on every collection.

In [ ]:
// for (i <- numbers) println(i)
numbers.foreach { i => println(i) }

var multiplied = collection.mutable.Buffer.empty[Int]
// for { i <- numbers.take(3)
//       j <- moreNumbers }
//   multiplied += i * j
numbers.take(3).foreach { i => 
  moreNumbers.foreach { j =>
    multiplied += i * j
  }
}

### Can we do this without a `var`?

`foreach` returns Unit, so a `for` loop, like the above, can only produce side effects.

- Printing is a side effect.
- Storing a computed value in a mutable buffer is a side effect.

By contrast, a `for` _comprehension_ is a form that can return, or _yield_, a value.

The general form is:

```
for <expressions> yield <expression>
```

Here are some examples:

In [ ]:
val doubled = for (num <- numbers) yield num * 2

val multiplied = for { 
  i <- numbers
  j <- moreNumbers } 
yield i * j

Not only is this approach better than manually appending to a mutable buffer, _it's easier to read_.

### Guard conditions

We can also filter our results _while_ looping:

In [ ]:
val evens = for (i <- numbers if i % 2 == 0) yield i

val multipliedOdds = for { i <- numbers if i % 2 != 0
                           j <- moreNumbers if j % 2 != 0} yield i * j

### `yield` can take a block

If you have to do more than a simple computation to yield a result, use a block:

In [ ]:
for (i <- numbers) yield {
  if (i % 2 == 0)
    i + 1
  else
    i + 3
}

### It's just more sweetness

Once again, this is just syntactic sugar for `map`, `flatMap` and `withFilter`.

(`withFilter` is just a special form of `filter`.)

Let's try to build a comprehension with `map`.

In [ ]:
val myFirstAttempt = numbers.take(3).map { i =>
  moreNumbers.map { j =>
    i * j
  }
}

Oh. That doesn't look right. We didn't get a single sequence back; we got a sequence of sequences.
Now we have to flatten it.

Hmm... Maybe we should just use `flatMap`:

In [ ]:
val mySecondAttempt = numbers.take(3).flatMap { i =>
  moreNumbers.map { j =>
    i * j
  }
}

So, multiple loop expressions in a `for` comprehension effectively behave like:

- `flatMap` calls for all expressions _except_ the last (inner-most) one
- a `map` call for the inner-most expression

### What about guards?

In [ ]:
val evens1 = for (i <- numbers if i % 2 == 0) yield i * 2

val evens2 = numbers.withFilter { i => i % 2 == 0 }.map { i => i * 2}

In [ ]:
val multOdds1 = for { 
  i <- numbers if i % 2 == 1
  j <- moreNumbers if j % 2 == 1 
} 
yield i * j

val multOdds2 = numbers.withFilter { i => i % 2 == 1 }.flatMap { i =>
  moreNumbers.withFilter { j => j % 2 == 1 }.map { j =>
    i * j
  }
}

println(multOdds1 == multOdds2)

### Recap

- `for` without `yield` is just syntactic sugar for one or more (nested) `foreach` calls.
- `for` _with_ `yield` is just syntactic sugar for a combination of `flatMap`, `map`, and (optionally) `withFilter`.
- Using `for` is often much more readable and more compact than manually composing `flatMap`, `map`, and `withFilter` (or `filter`) calls.

## Exercise 7.1

Convert the following to `for` comprehensions. Run the test cell (with the `assert` calls) to verify your solutions.

In [ ]:
val seq = 0 to 4

val result1a = seq.flatMap { i =>
  (0 to i by 3).withFilter { j => j % 2 == 1 }.map { j =>
    j + i
  }
}


In [ ]:
// Complete this for comprehension
val result1b = for { 
  /* fill this in */ 
}
yield /* fill this in */

In [ ]:
// ANSWER
val result1b = for {
  i <- seq
  j <- 0 to i by 3 if j % 2 == 1 
}
yield j + i

In [ ]:
val result2a = seq.flatMap(i => (i to 5)).withFilter(j => j % 3 != 1).map {j => j * 2}

In [ ]:
// Complete this for comprehension
val result2b = for { 
  /* fill this in */ 
} 
yield /* fill this in */

In [ ]:
// ANSWER
val result2b = for {
  i <- seq
  j <- i to 5 if j % 3 != 1
}
yield j * 2

In [ ]:
// TEST YOUR SOLUTIONS HERE

assert(result1a == result1b)
assert(result2a == result2b)

## You can pattern-match, too!

In [ ]:
val range = (0 to 2)

// Map the sequence into a sequence of 2-tuples
val seq1 = for {
  i1 <- range
  i2 <- range
}
yield (i1, i2)

// Watch how we destructure the 2-tuples while looping
val seq2 = for {
  (i1, i2) <- seq1 if ((i1 % 2) == 0 || (i2 % 2) == 1)
// ^^^^^^ destructuring via pattern matching
}
yield i1 + i2


## Temporary (local) variables

Consider this `for` comprehension:

In [ ]:
val seq1 = for {
  i1 <- range
  i2 <- range if (i1 + i2) % 2 == 0
}
yield i1 + i2

Note that we're calculating `i1 + i2` _twice_, which is inefficient.

This problem is simple to solve, though, with the addition of one more `for` feature:

In [ ]:
val seq2 = for {
  i1  <- range
  i2  <- range 
  sum  = i1 + i2
  if sum %2 == 0
}
yield sum

(Guard conditions don't need to be "attached to", or adjacent to, a loop expression.)

## `for` and `Option`

Because of the way `for` is desugared, it has some interesting properties. Consider the following:

In [ ]:
val ages = Map(
  "Joe" -> 12,
  "Joanna" -> 23,
  "Franklin" -> 43,
  "Amir" -> 38
)

// Recall that Map.get returns an option

val combinedAges = for {
  age1 <- ages.get("Joe")
  age2 <- ages.get("Joanna")
  age3 <- ages.get("Amir")
}
yield age1 + age2 + age3

It looks like the loop "unpacked" the `Option` values, then repacked them.

Implementing it by hand shows why that's the case. But recall what happens with `Option.map` first:

In [ ]:
val opt1: Option[String] = Some("10")
val opt2: Option[String] = None

val n1 = opt1.map(_.toInt)
val n2 = opt2.map(_.toInt)


In [ ]:
// Now, our comprehension, desugared:
ages.get("Joe").flatMap { age1 =>
  // We only get here if ages.get("Joe") returned a Some
  ages.get("Joanna").flatMap { age2 =>
    // Ditto.
    ages.get("Amir").map { age3 =>
      age1 + age2 + age3
    }
  }
}

What happens if one of the `get` calls returns a `None`?

In [ ]:
val combinedAges2 = for {
  age1 <- ages.get("Joe")
  age2 <- ages.get("Josephine")
  age3 <- ages.get("Amir")
}
yield age1 + age2 + age3

Wow! It's as if the `None` short-circuited the loop!

- This happens because the inner `flatMap` on `ages.get("Josephine")` (`None`) doesn't do anything.
- The remaining inner `map` call doesn't fire, because the `flatMap` on "Josephine" doesn't execute.

**For this to work, all the loops have to return the same type of "container".**

This pattern works with _many_ Scala types. Here are two common examples of types that are designed to work with
`for` comprehensions (and to "short-circuit"):

`scala.util.Try` 

- `Try` traps any exception in a block of code and returns it in a `Failure`. 
- It returns a `Success` containing the result of the block if no exception is thrown.
- for-comprehending over multiple `Try` values "short-circuits" on the first `Failure`, which is yielded back.




In [ ]:
import scala.util.Try

val try1 = for {
  i1 <- Try { "10".toInt }
  i2 <- Try { "20".toInt }
  i3 <- Try { "30".toInt }
}
yield i1 + i2 + i3

val try2 = for {
  i1 <- Try { "10".toInt }
  i2 <- Try { "20a".toInt }
  i3 <- Try { "30".toInt }
}
yield i1 + i2 + i3

`scala.concurrent.Future`

- Designed to run a block of code asynchronously.
- Contains a `Success` if the block completes.
- Contains a `Failure` on error.
- The first `Failure` short-circuits the loop.

In [ ]:
import scala.concurrent.{Future, Await}
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global

// Don't sleep in Futures! This is just for demo purposes.

val f1 = Future { Thread.sleep(500) ; 10 }
val f2 = Future { Thread.sleep(1000); 20 }
val f3 = Future { Thread.sleep(750); 30 }

// All three futures could be running right now.

val resF = for {
  i <- f1
  j <- f2
  k <- f3
}
yield i + j + k

// In a real application, you would never wait on a Future. You'd
// use its callback mechanism to fire more computation. We're waiting
// here, because it makes for a better demo.

val res = Await.ready(resF, 2.seconds)

In [ ]:
// Now a failure.

import scala.concurrent.{Future, Await}
import scala.concurrent.duration._
import scala.concurrent.ExecutionContext.Implicits.global

// Don't sleep in Futures! This is just for demo purposes.

val f1 = Future { Thread.sleep(500) ; 10 }
val f2 = Future { Thread.sleep(1000); throw new Exception("Math failure!") }
val f3 = Future { Thread.sleep(750); 30 }

// All three futures could be running right now.

val resF = for {
  i <- f1
  j <- f2
  k <- f3
}
yield i + j + k

// In a real application, you would never wait on a Future. You'd
// use its callback mechanism to fire more computation. We're waiting
// here, because it makes for a better demo.

val res = Await.ready(resF, 2.seconds)

![!](http://i.imgur.com/TvgcpDH.png) **Don't make this common mistake!**

If you code the above loop as follows, the futures don't all run in parallel. The first
future has to complete before the second one can start, etc.

```scala
val resF = for {
  i <- Future { ... }
  j <- Future { ... }
  k <- Future { ... }
}
yield i + j + k
```

![?](http://i.imgur.com/Guv4TBn.png) Why is that true?

## You can build your own

In [ ]:
object MyComprehendibleThing {
  
  class GuessingGame[A](values: Seq[A]) {
    def doYouHave(a: A): Boolean = values.contains(a)
    override val toString: String = s"GuessingGame($values)"
    
    def map[B](f: A => B) = new GuessingGame(values.map(f))
    def flatMap[B](f: A => Seq[B]) = new GuessingGame(values.flatMap(f))
    def withFilter(f: A => Boolean) = new GuessingGame(values.filter(f))
  }
  
  def apply(): Unit = {
    val game = new GuessingGame(Seq(1, 2, 3, 9, 21, 100))
    
    println(s"game=$game")
    println("have 21: " + game.doYouHave(21) + "; have 19: " + game.doYouHave(19))
    
    //I want a guessing game with double the values added
    //holding onto only even values but as Strings
    val newGame = for{
      i <- game
      j <- Seq(i, i * 2) if(0 == (j % 2))
    }
    yield{
      j.toString()
    }
    
    println(s"newGame=$newGame")
    println("have 42: " + newGame.doYouHave("42") + "; have 19: " + newGame.doYouHave("19"))
  }
}

MyComprehendibleThing()

## Summary

- `for` comprehensions make operating on collections more readable.
- `for` comprehensions are syntactic sugar for `map`, `flatMap`, `withFilter` (for `foreach`, if you don't use `yield`).
- Since they desugar to lambda functions, assignments and pattern matching can be used.
- Since `for` comprehensions are syntactic sugar for methods on classes:
    - We can use any class which defines these operations in a `for` comprehension.
    - We can define our own classes that can be used in `for` comprehensions.